# M - Automated Essay Scoring
_School of Information Technology_<br>
_Monash University Malaysia_<br>
(c) Copyright 2020, Ian Tan & Jun Qing Lim

Steps

- Read dataset (ASAP)
- Extract features (into file) using EASE
- Conduct machine learning (Sci-kit Learn libraries)
    - Naive Bayes
    - SVR
    - BLRR (later)
- Evaluate (QWK)

## Import Libraries

In [1]:
import time
start = time.time()

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm #SVR is in SVM
from sklearn.metrics import accuracy_score, confusion_matrix

### Import the EASE functions, which is located in the ease folder.

In [3]:
import sys
sys.path.insert(1, 'ease')
import create
import grade 
import model_creator 
import predictor_extractor 
import predictor_set 
import util_functions
import essay_set
import feature_extractor

from essay_set import EssaySet
from feature_extractor import FeatureExtractor

## Read Dataset

AES (Hewlett Foundation dataset from Kaggle) in the folder `asap-aes`.  For this, we use the `training_set_rel3` for training and testing.  Note that the `test_set` and the `valid_set` cannot be used as they don't contain the scores and are meant for the competition to score the entries.

In [4]:
data_set = pd.read_csv("asap-aes/training_set_rel3.tsv", sep='\t', encoding="latin-1")

In [5]:
data_set['essay'] = [entry.lower() for entry in data_set['essay']] # lower case for all words in essay

There are 8 different essay sets.  As an overview:
- Sets 1 & 2 are of persuasive/narrative in the form of letters
- Sets 3, 4, 5 & 6 are source dependent response to a given essay
- Sets 7 & 8 are of persuasive/narrative in the form of story writing essays

These format makes it good for transfer learning.

In [6]:
data_set_1 = data_set[data_set['essay_set'] == 1]
data_set_2 = data_set[data_set['essay_set'] == 2]
#data_set_3 = data_set[data_set['essay_set'] == 3]
#data_set_4 = data_set[data_set['essay_set'] == 4]
#data_set_5 = data_set[data_set['essay_set'] == 5]
#data_set_6 = data_set[data_set['essay_set'] == 6]
#data_set_7 = data_set[data_set['essay_set'] == 7]
#data_set_8 = data_set[data_set['essay_set'] == 8]

As each set will retain the original index, we want each of them to have their own indexing so that it is easier to match the essay and the scores.

In [7]:
data_set_1 = data_set_1.reset_index() # resets index
data_set_2 = data_set_2.reset_index()
#data_set_3 = data_set_3.reset_index()
#data_set_4 = data_set_4.reset_index()
#data_set_5 = data_set_5.reset_index()
#data_set_6 = data_set_6.reset_index()
#data_set_7 = data_set_7.reset_index()
#data_set_8 = data_set_8.reset_index()

We use just the `essay` content and the respective `scores`.

In [8]:
# If you want for the whole dataset.
# Commented out as we will work on individual datasets
#essays = data_set['essay']
#scores = data_set['domain1_score']

In [9]:
essays_1 = data_set_1['essay']
scores_1 = data_set_1['domain1_score']
essays_2 = data_set_2['essay']
scores_2 = data_set_2['domain1_score']
#essays_3 = data_set_3['essay']
#scores_3 = data_set_3['domain1_score']
#essays_4 = data_set_4['essay']
#scores_4 = data_set_4['domain1_score']
#essays_5 = data_set_5['essay']
#scores_5 = data_set_5['domain1_score']
#essays_6 = data_set_6['essay']
#scores_6 = data_set_6['domain1_score']
#essays_7 = data_set_7['essay']
#scores_7 = data_set_7['domain1_score']
#essays_8 = data_set_8['essay']
#scores_8 = data_set_8['domain1_score']

Rename the `domain1_score` column to `score`.

In [10]:
scores_1.columns = "score"
scores_2.columns = "score"
#scores_3.columns = "score"
#scores_4.columns = "score"
#scores_5.columns = "score"
#scores_6.columns = "score"
#scores_7.columns = "score"
#scores_8.columns = "score"

THE ABOVE NEEDS TO BE PUT INTO A LOOP BUT I LEFT IT AS IS BECAUSE YOU CAN PICK AND CHOOSE EASILY INSTEAD.

## Prepare Data

### Create the essay sets

Again, these can be looped but I kept them separated for ease of readability and commenting out those that we don't need.  Each set takes a long time to process, and hence please be patient with this part.

In [11]:
e_set_1 = EssaySet()
e_set_2 = EssaySet()
#e_set_3 = EssaySet()
#e_set_4 = EssaySet()
#e_set_5 = EssaySet()
#e_set_6 = EssaySet()
#e_set_7 = EssaySet()
#e_set_8 = EssaySet()

In [12]:
for i in range(len(essays_1)):
    e_set_1.add_essay(essays_1[i], scores_1[i])

In [13]:
for i in range(len(essays_2)):
    e_set_2.add_essay(essays_2[i], scores_2[i])

Left out for sets 3 - 6 for now.

In [14]:
"""
for i in range(len(essays_7)):
    e_set_7.add_essay(essays_7[i], scores_7[i])
"""

'\nfor i in range(len(essays_7)):\n    e_set_7.add_essay(essays_7[i], scores_7[i])\n'

In [15]:
"""
for i in range(len(essays_8)):
    e_set_8.add_essay(essays_8[i], scores_8[i])
"""

'\nfor i in range(len(essays_8)):\n    e_set_8.add_essay(essays_8[i], scores_8[i])\n'

## Extract Features

In [16]:
f_extractor = FeatureExtractor()

Change the next two variable assignment to change the evaluation of the essay sets.

Would be better to do this above.

**SETUP HERE**

In [17]:
e_set = e_set_2
score = scores_2


In [18]:
length = f_extractor.gen_length_feats(e_set)
length_df = pd.DataFrame(
    length, 
    columns = [
        'chars', 
        'words', 
        'commas', 
        'apostrophes', 
        'punctuations', 
        'avg_word_length',
        # new stuff, will need to compare original with new and separate punctuations
        'POS', 
        'POS/total_words'
    ]
)

_*Exclude the prompts for the time being*_

To be included next.

In [19]:
# Merge this with the score based on the index
# We use the shallow features first
features = length_df
dataset = features.merge(score, left_index=True, right_index=True)
dataset.columns = ['chars', 'words', 'commas', 'apostrophes', 'punctuations',
                   'avg_word_length',
                   'POS', 'POS/total_words', 'score']
#X_1 = dataset.iloc[:,0:10].values.astype(float)
#y_1 = dataset.iloc[:,11].values.astype(float)

## Determine Essay Prompts

In [20]:
essay_prompts = []

for i in range(1,9):
    file = "prompts/set" + str(i) + ".txt"
    f = open(file, "r", encoding="latin-1") # there are some 0x9x characters, hence need to specify encoding
    essay_prompts.append(f.read())
    
def get_essay_prompt(essay_set):
    return essay_prompts[essay_set-1]

In [21]:
len(essay_prompts)

8

**SETUP HERE**

In [22]:
# Unsure how this works
e_set.update_prompt(get_essay_prompt(2))

# Need more explanation on how this works - look into EASE
prompts = f_extractor.gen_prompt_feats(e_set)
prompts_df = pd.DataFrame(prompts, columns = [
    'prompt_words', 'prompt_words/total_words', 'synonym_words', 'synonym_words/total_words'
])
e_set # To check

In [23]:
# Another process that takes sometime to process
unstemmed = util_functions.get_vocab_essays_count(e_set._text, e_set._score)
stemmed = util_functions.get_vocab_essays_count(e_set._clean_stem_text, e_set._score)

bow = list(map(lambda a,b:[a,b], unstemmed, stemmed))
bow_df = pd.DataFrame(bow, columns = ['unstemmed', 'stemmed'])

In [24]:
features = pd.concat([length_df, prompts_df, bow_df], axis=1, sort=False)
features.head()

,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed
0,2639.0,527.0,15.0,13.0,21.0,5.007590,32.791587,0.062223,220.0,0.417457,112.0,0.212524,584,559
1,841.0,180.0,5.0,2.0,3.0,4.672222,17.866290,0.099257,82.0,0.455556,66.0,0.366667,210,210
2,1181.0,261.0,12.0,15.0,14.0,4.524904,22.171206,0.084947,144.0,0.551724,83.0,0.318008,291,285
3,2705.0,527.0,22.0,6.0,31.0,5.132827,7.026769,0.013334,245.0,0.464896,131.0,0.248577,547,528
4,2394.0,501.0,25.0,15.0,34.0,4.778443,31.795655,0.063464,216.0,0.431138,117.0,0.233533,591,562


In [25]:
# Export features to a file for next stage (optional)
dataset = features.merge(score, left_index=True, right_index=True)
dataset.head()

,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,domain1_score
0,2639.0,527.0,15.0,13.0,21.0,5.007590,32.791587,0.062223,220.0,0.417457,112.0,0.212524,584,559,4
1,841.0,180.0,5.0,2.0,3.0,4.672222,17.866290,0.099257,82.0,0.455556,66.0,0.366667,210,210,1
2,1181.0,261.0,12.0,15.0,14.0,4.524904,22.171206,0.084947,144.0,0.551724,83.0,0.318008,291,285,2
3,2705.0,527.0,22.0,6.0,31.0,5.132827,7.026769,0.013334,245.0,0.464896,131.0,0.248577,547,528,4
4,2394.0,501.0,25.0,15.0,34.0,4.778443,31.795655,0.063464,216.0,0.431138,117.0,0.233533,591,562,4


In [26]:
"""
dataset.columns = ['chars', 'words', 'commas', 'apostrophes', 'punctuations',
                   'avg_word_length', 'sentences', 'questions', 'avg_word_sentence',
                   'POS', 'POS/total_words',
                   'score']
"""

dataset.columns = ['chars', 'words', 'commas', 'apostrophes', 'punctuations',
                   'avg_word_length',
                   'POS', 'POS/total_words',
                   'prompt_words', 'prompt_words/total_words', 'synonym_words',
                   'synonym_words/total_words', 'unstemmed', 'stemmed',
                   'score']
dataset.head()
dataset.to_csv('maes_features.csv')

Can just use the features and score for the X and y but just to keep to certain convention if reading back from the CSV file above.

**YOU CAN RUN FROM HERE ON BY READING THE FEATURES FOR THE TRAINING**

In [4]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

In [5]:


def inverse_scale(test):
    return test * (6 - 1) + 1

In [6]:
cols_to_norm = ['chars', 'words', 'commas', 'apostrophes', 'punctuations', 'avg_word_length','POS','POS/total_words','prompt_words','prompt_words/total_words','synonym_words','synonym_words/total_words','score']
dataset[cols_to_norm] = dataset[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [7]:
dataset['stemmed'] = np.log(1 + dataset['stemmed'])
dataset['unstemmed'] = np.log(1 + dataset['unstemmed'])


In [8]:
dataset

,Unnamed: 0,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
0,0,0.413528,0.431718,0.208333,0.224490,0.032659,0.805052,0.465216,0.482895,0.317073,0.290328,0.298851,0.454360,6.371612,6.327937,0.6
1,1,0.112506,0.125991,0.069444,0.000000,0.004666,0.708027,0.253470,0.770305,0.106707,0.340438,0.166667,0.835261,5.351858,5.351858,0.0
2,2,0.169429,0.197357,0.166667,0.265306,0.021773,0.665406,0.314544,0.659249,0.201220,0.466925,0.215517,0.715020,5.676754,5.655992,0.2
3,3,0.424577,0.431718,0.305556,0.081633,0.048212,0.841284,0.099689,0.103477,0.355183,0.352722,0.353448,0.543450,6.306275,6.270988,0.6
4,4,0.372510,0.408811,0.347222,0.265306,0.052877,0.738757,0.451087,0.492528,0.310976,0.308321,0.313218,0.506276,6.383507,6.333280,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,0.195881,0.219383,0.000000,0.653061,0.017107,0.710803,0.452443,0.865379,0.155488,0.265527,0.201149,0.603128,5.717028,5.683580,0.4
1796,1796,0.201574,0.215859,0.208333,0.122449,0.023328,0.764897,0.180996,0.351097,0.213415,0.450199,0.206897,0.630213,5.828946,5.811141,0.4
1797,1797,0.072995,0.083700,0.097222,0.000000,0.007776,0.682310,0.048053,0.199138,0.089939,0.448715,0.074713,0.565688,5.105945,5.081404,0.2
1798,1798,0.429935,0.461674,0.305556,0.061224,0.041991,0.767788,0.284762,0.277669,0.440549,0.446957,0.459770,0.669200,6.297109,6.240276,0.4


Reshape the data and label

In [9]:
X = dataset.iloc[:,1:15].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X

array([[0.41352754, 0.43171806, 0.20833333, ..., 0.45436   , 6.37161185,
        6.32793678],
       [0.11250628, 0.12599119, 0.06944444, ..., 0.8352608 , 5.35185813,
        5.35185813],
       [0.1694291 , 0.19735683, 0.16666667, ..., 0.71502012, 5.6767538 ,
        5.65599181],
       ...,
       [0.07299514, 0.08370044, 0.09722222, ..., 0.56568756, 5.10594547,
        5.08140436],
       [0.42993471, 0.46167401, 0.30555556, ..., 0.66920016, 6.29710932,
        6.24027585],
       [0.38389419, 0.42555066, 0.11111111, ..., 0.71329043, 6.2441669 ,
        6.20859003]])

In [10]:
X.shape

(1800, 14)

In [11]:
y = np.array(y).reshape(-1,1)
y.shape

(1800, 1)

#### K Fold CV

In [12]:
from sklearn.model_selection import KFold

In [13]:
kf = KFold(n_splits=5, random_state=13, shuffle=True)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):
    ## print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]


In [14]:
print(y_test[:5, :])

[[0. ]
 [0.6]
 [0.4]
 [0.4]
 [0.4]]


## Model Training

In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### SVM Training

Use standard scaler for the data

In [16]:
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_train,y_train.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

At this stage, the Support Vector Machine (SVM) model is called `model_svm`

### BLRR

In [17]:
from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_train, y_train.ravel())

BayesianRidge()

At this stage, the Bayesian Linear Ridge Regression (BLRR) model is called `model_blrr_1`

## Prediction

We will be using the respective validation set and will have to also pre-process the data.

### SVM

In [18]:
y_predSVM = model_svm.predict(X_test)
y_predSVM = inverse_scale(y_predSVM).round()

cm = confusion_matrix(inverse_scale(y_test).round(), y_predSVM)
#np.set_printoptions(threshold=np.inf)
print(cm)

[[  0   2   0   1   0   0]
 [  3  10  16   0   0   0]
 [  0   0 116  37   0   0]
 [  0   0  37 121   1   0]
 [  0   0   1  12   2   0]
 [  0   0   0   1   0   0]]


### BLRR

In [21]:
y_predBLRR = model_blrr.predict(X_test)
y_predBLRR = inverse_scale(y_predBLRR).round()

cm = confusion_matrix(inverse_scale(y_test).round(), y_predBLRR)
print(cm)

[[  1   1   0   1   0   0]
 [  3  11  15   0   0   0]
 [  0   0 121  31   1   0]
 [  0   0  39 117   3   0]
 [  0   0   1  11   3   0]
 [  0   0   0   0   1   0]]


### Ensembling the 3 Algorithms

In [22]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
results

NameError: name 'y_predNB' is not defined

In [32]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results

,Actual,NB,SVM,BLRR,Ensemble
0,1.0,3.0,2.0,2.0,2.0
1,4.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,3.0,5.0,4.0,4.0,4.0
4,3.0,4.0,4.0,3.0,4.0
...,...,...,...,...,...
355,4.0,4.0,4.0,4.0,4.0
356,4.0,4.0,4.0,4.0,4.0
357,3.0,3.0,4.0,4.0,4.0
358,4.0,4.0,4.0,4.0,4.0


## Evaluation using QWK

QWK scores for NB, SVR and BLRR

In [23]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

### Naive Bayes

In [34]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.09      0.33      0.14         3
         2.0       0.29      0.21      0.24        29
         3.0       0.60      0.63      0.61       153
         4.0       0.64      0.51      0.57       159
         5.0       0.16      0.27      0.20        15
         6.0       0.06      1.00      0.12         1

    accuracy                           0.53       360
   macro avg       0.31      0.49      0.31       360
weighted avg       0.57      0.53      0.54       360



In [35]:
print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

0.538569524836475


### SVM

In [25]:
rpt = classification_report(inverse_scale(y_test).round(), y_predSVM)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         3
         2.0       0.83      0.34      0.49        29
         3.0       0.68      0.76      0.72       153
         4.0       0.70      0.76      0.73       159
         5.0       0.67      0.13      0.22        15
         6.0       0.00      0.00      0.00         1

    accuracy                           0.69       360
   macro avg       0.48      0.33      0.36       360
weighted avg       0.70      0.69      0.68       360



C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print(cohen_kappa_score(inverse_scale(y_test).round(), y_predSVM, weights="quadratic"))

0.6295686532762594


### BLRR

In [38]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         3
         2.0       0.80      0.41      0.55        29
         3.0       0.65      0.82      0.72       153
         4.0       0.75      0.65      0.69       159
         5.0       0.40      0.40      0.40        15
         6.0       0.00      0.00      0.00         1

    accuracy                           0.68       360
   macro avg       0.43      0.38      0.39       360
weighted avg       0.69      0.68      0.67       360



In [28]:
print(cohen_kappa_score(inverse_scale(y_test).round(), y_predBLRR, weights="quadratic"))

0.6571590479788441


### Ensemble

In [114]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         3
         2.0       0.70      0.42      0.53        33
         3.0       0.66      0.76      0.71       151
         4.0       0.69      0.75      0.72       154
         5.0       1.00      0.06      0.11        18
         6.0       0.00      0.00      0.00         1

    accuracy                           0.68       360
   macro avg       0.51      0.33      0.34       360
weighted avg       0.69      0.68      0.66       360



In [35]:
print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

0.6962879640044994


In [58]:
end = time.time()
print("Total time to execute the notebook is " + str(end - start))

Total time to execute the notebook is 271.73689579963684


QWK scores output are from -1 to 1, where -1 means that it is totally wrong while 1 is a perfect match (classification).  The aim is to get as close as possible to 1, with a score of 0.6 being generally accepted as a good score.

On the output of the QWK agreements, the score is just "moderate agreement".  Work now is to achieve substantial agreement.

https://www.statisticshowto.com/cohens-kappa-statistic/

In short, BLRR works better than SVM but a small margin but better than NB.

## Appendix

### QWK Scores (Manual Code)

In [ ]:
N = len(cm) # Just to get the same size as the confusion matrix from above
w = np.zeros((N,N)) # create a matrix of N by N
d = (N-1)**2 # the weighted portion
for i in range(len(w)):
    for j in range(len(w)):
        w[i][j] = float(((i-j)**2)/d) 
w # The weighted matrix

In [ ]:
N

In [ ]:
np.unique(y_test)

In [ ]:
np.unique(y_predNB)

In [ ]:
act_hist=np.zeros([N])
for item in y_test: 
    act_hist[item-1] += 1

In [ ]:
pred_hist=np.zeros([N])
for item in y_predNB: 
    pred_hist[item-1]+=1

In [ ]:
E = np.outer(act_hist, pred_hist)
E

In [ ]:
E = E/E.sum()
E.sum()

In [ ]:
cm = cm/cm.sum()
cm.sum()

In [ ]:
num=0
den=0
for i in range(len(w)):
    for j in range(len(w)):
        num+=w[i][j]*cm[i][j]
        den+=w[i][j]*E[i][j]
            
weighted_kappa = (1 - (num/den))
weighted_kappa

# END

In [ ]:
#length

Naive Bayes: 0.44386909693454846

SVM: 0.564516129032258

BLRR: 0.6013422818791946

Ensemble: 0.561464569121537
    
#pos
Naive Bayes: 0.5109078114004222

SVM: 0.582941571524513

BLRR: 0.6172680412371134

Ensemble: 0.6037698930341768
    
#bow
Naive Bayes: 0.5349235893416928

SVM: 0.5708529214237743

BLRR: 0.5922570016474464

Ensemble: 0.5923591965340684
    
    
#prompt

Naive Bayes: 0.2804039837284331

SVM: 0.5770676691729324

BLRR: 0.5710059171597632

Ensemble: 0.5485678704856787
    

In [ ]:
#BOW
Naive Bayes: 0.0

SVM: 0.5903066006158789

BLRR: 0.5738161559888579

Ensemble: 0.5694520925110131
    
#POS 
Naive Bayes: 0.4830633284241532

SVM: 0.5837900794738284

BLRR: 0.5363636363636364

Ensemble: 0.5571245186136072
    
#length
Naive Bayes: 0.504200146092038

SVM: 0.5801449859041481

BLRR: 0.6036624002027626

Ensemble: 0.598688524590164
    
    #prompt
Naive Bayes: 0.24556868537666177

SVM: 0.5690834473324213

BLRR: 0.5431773236651285

Ensemble: 0.5426452410383189

In [75]:
0.517 - 0.246 

0.271

In [76]:
0.584 - 0.569 

0.015000000000000013

In [77]:
0.620 - 0.543

0.07699999999999996

In [78]:
0.613 - 0.542

0.07099999999999995

In [79]:
0.271+0.015+0.077+0.071

0.43400000000000005